# writing strings to files
This is the most flexible, but unstructured, way to save data.  Unless you're really only saving a text file, you're going to have to come up with some way to encode your data.  There are better options for that.

In [1]:
import os

out_data = "I want to save this text to a file."

# without a directory, this will go into your current working directory
file_name = 'test_file.txt'

# write it out
with open(file_name, 'w') as f:
    f.write(out_data)

# read it back in
with open(file_name, 'r') as f:
    in_data = f.read()
    
print in_data

I want to save this text to a file.


In [2]:
# i want to save these to separate files
out_data = ['string1', 'string2', 'string3' ]

for i, d in enumerate(out_data):
    # let's make a unique file name for each string
    file_name = 'test_file_%d.txt' % i
    
    with open(file_name, 'w') as f:
        f.write(d)    
        print "saving '%s' to '%s'" % (d, file_name)

saving 'string1' to 'test_file_0.txt'
saving 'string2' to 'test_file_1.txt'
saving 'string3' to 'test_file_2.txt'


# CSV files
The simplest way to store tabular data.  If your data structures are flat (not nested) and not too big, go ahead and use this.

In [3]:
import csv

# let's save these to a csv
data = [ [ 'jim', 10 ], [ 'bob', 20 ], [ 'jill', 30 ] ]

file_name = 'test_file.txt'

# write it out
with open(file_name, 'w') as f:
    writer = csv.writer(f, delimiter=',')
    
    # header row
    writer.writerow(['name', 'age'])
    
    for d in data:
        writer.writerow(d)
        
# read it back in
with open(file_name, 'r') as f:
    reader = csv.reader(f)
    for row in list(reader):
        print row

['name', 'age']
['jim', '10']
['bob', '20']
['jill', '30']


In [4]:
import csv

# let's save these dictionaries to a csv
data = [ { 'name': 'jim', 'age': 10 }, 
         { 'name': 'bob', 'age': 20 }, 
         { 'name': 'jill', 'age': 30 } ]

file_name = 'test_file.csv'

# write it out
with open(file_name, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['name','age'])
    
    writer.writeheader()
    
    for d in data:
        writer.writerow(d)
        
# read it back in
with open(file_name, 'r') as f:
    reader = csv.DictReader(f)
    for row in list(reader):
        print row

{'age': '10', 'name': 'jim'}
{'age': '20', 'name': 'bob'}
{'age': '30', 'name': 'jill'}


**warning**: CSV returns everything as strings!

In [5]:
file_name = 'test_file.csv'

# read it back in
with open(file_name, 'r') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

print "ick", rows[0]['age'] + rows[1]['age'] 
print "whew", int(rows[0]['age']) + int(rows[1]['age'])


ick 1020
whew 30


If you are used to thinking in R (or Pandas) data frames, use the Pandas importer.  It takes care of data type conversions for you.

In [6]:
import pandas as pd

data = pd.DataFrame.from_csv('test_file.csv')
data

age
name     
jim    10
bob    20
jill   30

# JSON: saving generic dictionaries and lists
If your data structures are not flat, CSV will be a bit of a hassle.  More generic storage of dictionaries and lists can be done with JSON.

In [7]:
import json

data = {
    'name': 'bob',
    'age': 40,
    'children': [ { 'name': 'suzanne', 'age': 10 }, 
                  { 'name': 'artichoke', 'age': 3 } ]        
    }

file_name = 'test_file.json'     

with open(file_name, 'w') as f:
    json.dump(data, f, indent=2)

with open(file_name, 'r') as f:
    in_data = json.load(f)

print in_data

{u'age': 40, u'name': u'bob', u'children': [{u'age': 10, u'name': u'suzanne'}, {u'age': 3, u'name': u'artichoke'}]}


**Warning!**  The json package that comes with Python doesn't like numpy data types, or really any data types that aren't standard Python.  We've added a utility function to the AllenSDK to help with saving these out.

In [8]:
import json
import numpy as np

# the allensdk has code for dealing with this
import allensdk.core.json_utilities as json_utilities

data = {
    'name': 'bob',
    'age': np.float32(40), # uh oh! json doesn't like numpy data types
    'children': [ { 'name': 'suzanne', 'age': 10 }, 
                  { 'name': 'artichoke', 'age': 3 } ]        
    }

file_name = 'test_file.json'     

json_utilities.write(file_name, data)

in_data = json_utilities.read(file_name)

print in_data

{u'age': 40.0, u'name': u'bob', u'children': [{u'age': 10, u'name': u'suzanne'}, {u'age': 3, u'name': u'artichoke'}]}


# string munging
Python is really good at slicing and dicing strings.  

In [9]:
import math

strings = [ 'some', 'strings', 'please' ]

# join
joined = ' '.join(strings)
print "joined:", joined

# split
print "split:", joined.split()

# formatting
print 'my_file_name_%02d_%s.txt' % (3, 'suffix')
print 'pi: %f' % math.pi
print 'shorter pi: %.4f' % math.pi

# endswith, startswith
print 'file_name.txt'.endswith('txt')
print '/home/davidf/file_name.txt'.startswith('/home/')

joined: some strings please
split: ['some', 'strings', 'please']
my_file_name_03_suffix.txt
pi: 3.141593
shorter pi: 3.1416
True
True


For more sophisticated string management, use regular expressions.  The syntax is a little confusing, but it has some nice features.

In [10]:
import re

# look for non-whitespace characters -> underscore -> digits -> anything
m = re.match('(\S+)_(\d+).*', 
             'file_12.txt')

# parentheses in the pattern will be stored in match groups
print m.groups()

# careful about indexing
print 'total match', m.group(0)
print 'group 1', m.group(1)
print 'group 2', m.group(2)

('file', '12')
total match file_12.txt
group 1 file
group 2 12


This is really just scratching the surface of regular expressions.  It's a bit of a rabbit hole, so be careful.

# files, directories

In [11]:
import os

# where am I?
print "cwd:", os.getcwd()

# what is this file's extension?
file_name = 'file_name.txt'
print "splitext:", os.path.splitext(file_name)

# what is the absolute path of this relative file name?
abs_file_name = os.path.abspath(file_name)
print "absolute:", abs_file_name

# what directory is this file in?
print "directory:", os.path.dirname(abs_file_name)

# what is the name of this file, minus all the other stuff?
print "base name:", os.path.basename(abs_file_name)         

# what files are in my cwd?
print os.listdir(os.getcwd())


cwd: c:\workspace\SWDB-2015\presentations\python\tutorial_part_2
splitext: ('file_name', '.txt')
absolute: c:\workspace\SWDB-2015\presentations\python\tutorial_part_2\file_name.txt
directory: c:\workspace\SWDB-2015\presentations\python\tutorial_part_2
base name: file_name.txt
['.ipynb_checkpoints', 'f1_scripts.py', 'f2_modules.py', 'functions_notebook.ipynb', 'numpy_notebook.ipynb', 'random_data.h5', 'random_data.mat', 'random_data.npy', 'random_data.npz', 'scripts_and_modules_notebook.ipynb', 'strings_and_files.ipynb', 'submodule', 'test_file.csv', 'test_file.json', 'test_file.txt', 'test_file_0.txt', 'test_file_1.txt', 'test_file_2.txt']


# Writing arrays to files

In [12]:
import numpy as np
import scipy.io
import h5py

data = np.random.random((100,100))

# save it as a numpy file
np.save('random_data.npy', data)
data = np.load('random_data.npy')

# save it as a compressed numpy file (if it's big)
np.savez('random_data.npz', data=data)
mdict = np.load('random_data.npz')
data = mdict['data']

# save it as a mat file
scipy.io.savemat('random_data.mat', { 'data': data })
mdict = scipy.io.loadmat('random_data.mat')
data = mdict['data']

# save it in an hdf5 file
f = h5py.File('random_data.h5', 'w')
f['data'] = data
f.close()
f = h5py.File('random_data.h5', 'r')
data = f['data'].value